
#Economic News Sentiment Classifier

## Author

Anael Umar

**GitHub**: [Anael-UmarDS](https://github.com/Anael-UmarDS)

---

## Description

This project pulls **real-time economic headlines** from [NewsAPI](https://newsapi.org) and automatically labels them using **VADER sentiment analysis**. It then trains a **logistic regression classifier** to categorize future headlines as **positive**, **neutral**, or **negative**. The final output includes evaluation metrics and trend **plots** to visualize **sentiment shifts** over time.

---

## Objectives

- Applying machine learning to understand **live financial data**
- Automate labeling of economic headlines using **natural language processing (NLP)** tools
- **Visualize** economic sentiment trends over a strict period of time
- Provide a **foundation** for further research in economic forecasting and AI-based sentiment analysis
- **Applying machine learning** to understand the economy at a **macroeconomic level**

---

## How It Works

1. **Data Fetching**  
   Headlines are fetched using the NewsAPI for the keyword **economy**.

2. **Auto-labeling with VADER**  
   Each headline is scored using VADER sentiment analysis and categorized into:
   - **positive**
   - **neutral**
   - **negative**

3. **Training a Classifier**  
   A logistic regression model is trained on the auto-labeled data using TF-IDF features.

4. **Evaluation**  
   The model is evaluated using accuracy, precision, recall, F1-score, and a confusion matrix.

5. **Visualization**  
   Plots are generated to track:
   - Percentage of each sentiment by week
   - Sentiment trends over time for economic headlines

---

## Example Use Cases

- Predicting **market mood shifts** based on economic news sentiment
- **Tracking** government or policy-related economic tone
- Training ML pipelines with **auto-labeled** real-world data
- Demonstration of **applied machine learning** and **NLP**

---

## Visual Outputs

- **Confusion Matrix**
- **Line Charts** of Sentiment Percentage per Week  
- Real world **demo headlines** pulled from Google News for testing

---
## Limitations

- **Headline Context**: Headlines lack full article context, which could influence sentiment accuracy
- **Bias**: Certain media sources may exhibit political or economic bias that skews sentiment labeling
- **Imbalanced Labels**: Economic headlines are often skewed toward neutral sentiment, which can affect model generalization
- **Query Limit for Free Developer Plan**: The API free Developer plan only allows up to 100 requests per day and only articles up to a month old, therefore forcing training data to be short and visualization to plot weekly headline percentage over a total of 1 month
---

## Setup Instructions

1. Get your free NewsAPI key by going to https://newsapi.org or clicking the link above in **Description**
2. Click **"Get API Key"** and register with your information
3. Replace your key in the cell under **API Setup** with where it says **"YOUR KEY HERE"**
4. Run the notebook in Jupyter Labs, Google Colab, or in a Python file

# Importing Libraries

In [153]:
# Import required libraries

import requests                                                                                 # Allows for HTTP requests to be sent from Python
import pandas as pd                                                                             # Used for data manipulation and analysis
import numpy as np                                                                              # Used to store data in multi-dimensional arrays
import matplotlib.pyplot as plt                                                                 # Used to illustrate data as plots or graphs
import joblib                                                                                   # Used to run large computations and efficiently handle them
from collections import Counter                                                                 # Used to iterate through hashable objects
from typing import List                                                                         # Used to indicate the expected types of variables, function parameters, and return values
from nltk.sentiment import SentimentIntensityAnalyzer                                           # Give a sentiment intensity score to sentences
from sklearn.feature_extraction.text import TfidfVectorizer                                     # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.linear_model import LogisticRegression                                             # Used for multi-class classification problems
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix             # Used for accuracy of a classification model, text report showing the main classification metrics for each class, computing a confusion matrix, respectively
from sklearn.model_selection import train_test_split                                            # Split arrays or matrices into random train and test subsets
import nltk                                                                                     # Used for working with Natural Language Processing (NLP)
from textblob import TextBlob                                                                   # Used for processing textual data

nltk.download("vader_lexicon")                                                                  # Using NLTK Library to download VADER, a sentiment analysis tool

# API Setup

In [135]:
NEWSAPI_KEY = "YOUR KEY HERE"                                                                   # API Key
NEWSAPI_ENDPOINT = "https://newsapi.org/v2/everything"                                          # Point of Contact for API
RANDOM_STATE = 42                                                                               # Ensures Reproducibility
LABELS = ["positive", "neutral", "negative"]                                                    # Labels for Classification

# Fetching and Labeling Data

In [136]:
# Connect to the API and retrieve the data
# (headlines related to the economy and when they were published)
# and store in Pandas DataFrame returning headlines column
# Define parameters and return type using typing library

def fetch_news_data(query="economy", page_size=100) -> pd.DataFrame:

    # Access API parameters/necessities
    params = {
        "q": query,
        "language": "en",
        "pageSize": page_size,
        "apiKey": NEWSAPI_KEY
    }

    # Access HTTP server to access API
    response = requests.get(NEWSAPI_ENDPOINT, params=params)
    data = response.json()

    # Handle Exceptions
    if response.status_code != 200 or "articles" not in data:
        raise ValueError("Failed to fetch news data from NewsAPI")

    # Create headline column for Pandas DataFrame
    headlines = [
      {
          "headline": article["title"],
          "publishedAt": article.get("publishedAt")
      }
      for article in data["articles"]
      if article["title"]
    ]

    return pd.DataFrame(headlines)

In [137]:
# Label returned Pandas DataFrame of fetch_news_data with labels
# based on sentiment value provided by NLTK VADER
# and return it as a new Pandas DataFrame
# Define parameters and return type using typing library

def auto_label_data(df: pd.DataFrame) -> pd.DataFrame:

    # Create labeled_rows column for Pandas DataFrame
    labeled_rows = []
    for i, row in df.iterrows():
        headline = row["headline"]
        polarity = TextBlob(headline).sentiment.polarity

        # Auto-label based on polarity
        if polarity > 0.1:
            label = "positive"
        elif polarity < -0.1:
            label = "negative"
        else:
            label = "neutral"

        # Extract date from publishedAt
        date_str = str(row.get("publishedAt", ""))[:10]
        try:
            date_obj = pd.to_datetime(date_str)
            week = date_obj.to_period("W").start_time.date()
            month = date_obj.strftime("%Y-%m")
        except Exception:
            week = pd.NaT
            month = None

        labeled_rows.append({
            "headline": headline,
            "sentiment": label,
            "week": week,
            "date": month
        })

    return pd.DataFrame(labeled_rows)

# Cleaning and Preparing Data for Training and Evaluation

In [138]:
# Clean data to lowercase and no blanks
# Helper function

def basic_text_clean(text: str) -> str:
    return text.lower().strip()

In [139]:
# Clean Pandas DataFrame using helper function

def prepare_features(df: pd.DataFrame):
    X_raw = df["headline"].apply(basic_text_clean)
    y = df["sentiment"]
    return X_raw, y

# Train and Evaluate Data

In [140]:
# Training function to train model to classify headlines
# TF-IDF (Term Frequency - Inverse Document Frequency)
# TF: Term Frequency: number of times the term appears in a document compared to the total number of words in the document
# IDF: Inverse Document Frequency: ratio of total number of documents to number of documents containing the term
# Uses TF-IDF Sentiment Weightage System: TF-IDF = TF * IDF
# Low TF-IDF indicates high frequency of a term
# Logistic regression: statistical analysis method used to predict multinomial outcome

def train_model(X_train, y_train):
    vectoriser = TfidfVectorizer(max_features=6000, ngram_range=(1, 2), stop_words="english")                                   # Convert text into numerical features
    X_train_vec = vectoriser.fit_transform(X_train)                                                                             # Fit the vectorizer to the training text data (X_train) and then transform it (evaluating how important word is to document)
    model = LogisticRegression(max_iter=2000, n_jobs=-1, class_weight="balanced")                                               # Initialize a Logistic Regression model for classification
    model.fit(X_train_vec, y_train)                                                                                             # Train the Logistic Regression model using the vectorized training features (X_train_vec) and their corresponding labels (y_train)
    return model, vectoriser                                                                                                    # Return trained model and numerical features

In [141]:
# Predicts label for each headline based on sentiment value

def predict(model, vectoriser, headlines: List[str]):
    cleaned = [basic_text_clean(h) for h in headlines]
    return model.predict(vectoriser.transform(cleaned))

In [142]:
# Evaluation of the model by calculating the accuracy of predictions and calling plot_confusion_matrix
# Calculates precision, recall, and trade-off rate of precision and recall (f1-score)

def evaluate_model(model, vectoriser, X_test, y_test):
    X_test_vec = vectoriser.transform(X_test)
    y_pred = model.predict(X_test_vec)
    print("\nEvaluation:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}\n")
    print(classification_report(y_test, y_pred, digits=3))
    cm = confusion_matrix(y_test, y_pred, labels=LABELS)
    plot_confusion_matrix(cm)

# Visualizing Data

In [143]:
# Plots confusion matrix that compares true labels vs assigned labels
# And saves as an image called "confusion_matrix.png"

def plot_confusion_matrix(cm: np.ndarray):
    fig, ax = plt.subplots(figsize=(6, 4))
    im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(len(LABELS)),
           yticks=np.arange(len(LABELS)),
           xticklabels=LABELS,
           yticklabels=LABELS,
           ylabel="True label",
           xlabel="Predicted label",
           title="Confusion Matrix")
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], "d"), ha="center", va="center", color="black")
    fig.tight_layout()
    plt.savefig("confusion_matrix.png", dpi=300)
    plt.close()
    print("Confusion matrix saved → confusion_matrix.png")

In [144]:
# Plots each sentiment percentage of total headlines per week
# As three separate line trends where the x-axis is each week and the y-axis is the percentage of each sentiment

def plot_sentiment_percentage_trend(labeled_df: pd.DataFrame):

    # Ensure 'week' is datetime
    labeled_df["week"] = pd.to_datetime(labeled_df["week"])

    # Count sentiment per week
    weekly_counts = labeled_df.groupby(["week", "sentiment"]).size().unstack(fill_value=0)

    # Full weekly range from start to end of dataset
    full_weeks = pd.date_range(
        start=labeled_df["week"].min(),
        end=labeled_df["week"].max(),
        freq="W-MON"
    )

    # Reindex to include all weeks
    weekly_counts = weekly_counts.reindex(full_weeks, fill_value=0)
    weekly_counts.index.name = "week"

    # Calculate total per week and derive percentages
    weekly_totals = weekly_counts.sum(axis=1)
    sentiment_percentages = weekly_counts.divide(weekly_totals, axis=0).fillna(0) * 100

    # Plotting
    plt.figure(figsize=(12, 6))
    for sentiment in ["positive", "neutral", "negative"]:
        if sentiment in sentiment_percentages.columns:
            plt.plot(sentiment_percentages.index, sentiment_percentages[sentiment], label=sentiment, marker="o")

    plt.title("Weekly Percentage of Economic Headlines by Sentiment")
    plt.xlabel("Week")
    plt.ylabel("Sentiment Share (%)")
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("line_trend.png", dpi=300)
    plt.close()
    print("Line trend saved → line_trend.png")
    #plt.show()


In [145]:
# Efficiently save the models using joblib

def save_artifacts(model, vectoriser):
    joblib.dump(model, "sentiment_model.joblib")
    joblib.dump(vectoriser, "tfidf_vectorizer.joblib")
    print("Model artifacts saved.")

# Main Function

In [150]:
# Main Running Function
def main():

    # Number of queries to fetch
    n = 100

    # Fetch query from API (see function documentation comments on how)
    news_df = fetch_news_data(page_size=n)

    # Label query from API (see function documentation comments on how)
    labeled_df = auto_label_data(news_df)

    # Print how many headlines are of each sentiment based on VADER sentiment analysis
    print("\nClass distribution:", dict(Counter(labeled_df["sentiment"])))

    # Clean data and prepare for training
    # Implement stratified random sampling
    # Stratified random sampling splits data into subsets
    # and randomly chooses each from there until None are left
    X, y = prepare_features(labeled_df)
    min_class_size = min(Counter(y).values())
    can_stratify = min_class_size >= 2

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y if can_stratify else None, random_state=RANDOM_STATE
    )

    # Train model (see function documentation comments on how)
    model, vectoriser = train_model(X_train, y_train)

    # Evaluate model (see function documentation comments on how)
    evaluate_model(model, vectoriser, X_test, y_test)

    # Save models using save_artifact (see function documentation comments on how)
    save_artifacts(model, vectoriser)

    # List of real demo headlines pulled at random from Google News
    demo_headlines = [
      "The S&P 500 Is Projected to Rally More Than Expected",
      "What Division Inside the Fed Means for Future Interest-Rate Cuts",
      "Citing trade wars, the World Bank sharply downgrades global economic growth forecast to 2.3%",
      "Trump Threatens 30% Tariffs on EU, Mexico",
      "Faisal Islam: We are heading for significant tax rises",
      "Reeves disappointed after economy unexpectedly shrinks",
    ]

    # Print results
    print("\nDemo predictions:")
    for h, p in zip(demo_headlines, predict(model, vectoriser, demo_headlines)):
        print(f"[{p}] {h}")

# Running Main

In [152]:
# Run main()
if __name__ == "__main__":
    main()

## Conclusions

- **Sentiment classification** on news headlines is feasible using lightweight models
- **Weekly sentiment trends** can uncover **mood shifts** in the economy
- Even rule-based systems like **VADER** can support training useful **supervised models**
- The classifier achieved **reasonable performance** in identifying headline sentiment automatically

---

## Implications

###Financial Markets

- **Investor Sentiment Monitoring**: Investigate market mood in real-time
- **Event-Driven Trading**: Identify spikes in optimism or fear to adjust strategy

###Public Policy

- **Macroeconomic Tracking**: Understand how the media presents economic news
- **Crisis Detection**: Spot growing negative trends in headlines

###ML Development

- **Semi-Supervised Learning**: Demonstrates how to bootstrap datasets from unlabeled text
- **Fine-Tuning Opportunity**: Base model could be improved using deep learning

---

## Machine Learning Concepts

- API
- Natural Language Processing (NLP)
- Feature Engineering
- Model Training & Evaluation
- Data Visualization